In [13]:
import pyscheduling_cc.FS.FmCmax as fm_cmax
from pyscheduling_cc.FS.FlowShop import Job, FlowShopSolution

from docplex.cp.model import CpoModel
from docplex.cp.solver.cpo_callback import CpoCallback
from docplex.cp.expression import INTERVAL_MAX

In [34]:
def csp_transform_solution(msol, E_i, instance ):

    sol = FlowShopSolution(instance)
    for k in range(instance.m):
        k_tasks = []
        for i in range(instance.n):
            start = msol[E_i[i][k]][0]
            end = msol[E_i[i][k]][1]
            k_tasks.append(Job(i,start,end))

            k_tasks = sorted(k_tasks, key= lambda x: x[1])
            sol.machines[k].job_schedule = [job[0] for job in k_tasks]
    
    sol.job_schedule = sol.machines[0].job_schedule
    
    return sol

In [45]:
instance = fm_cmax.FmCmax_Instance.generate_random(10,3)
E = range(instance.n)
M = range(instance.m)

model = CpoModel("FS_Model")

E_i = [[] for i in E]
M_k = [[] for k in M]
types_k = [ list(range(instance.n)) for k in M ]
for i in E:
    for k in M:
        start_period = (instance.R[i], INTERVAL_MAX) if hasattr(instance, 'R') else (0, INTERVAL_MAX)
        job_i = model.interval_var( start = start_period,
                                    size = instance.P[i][k], optional= False, name=f'E[{i},{k}]')
        E_i[i].append(job_i)
        M_k[k].append(job_i)

# No overlap inside machines
seq_array = []
for k in M:
    seq_k = model.sequence_var(M_k[k], types_k[k], name=f"Seq_{k}")
    seq_array.append(seq_k)
    
    model.add( model.no_overlap(seq_k) )

# Same sequence constraint
for k in range(1, instance.m):
    model.add( model.same_sequence(seq_array[k - 1], seq_array[k]) )

# Precedence constraint between machines for each job
for i in E:
    for k in range(1, instance.m):
        model.add( model.end_before_start(E_i[i][k - 1], E_i[i][k]) )

# Add objective
model.add( model.minimize( model.max(model.end_of(job_i) for i in E for job_i in E_i[i]) ) )

In [46]:
msol = model.solve(LogVerbosity="Normal", Workers=1, TimeLimit=30, LogPeriod=1000000,
                   log_output=True, trace_log=True, add_log_to_solution=True, RelativeOptimalityTolerance=0)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Minimization problem - 33 variables, 25 constraints
 ! TimeLimit            = 30
 ! Workers              = 1
 ! LogPeriod            = 1000000
 ! RelativeOptimalityTolerance = 0
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 147.2 (before), 147.2 (after)
 !  . Memory usage      : 476.9 kB (before), 476.9 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         33                 -
 + New bound is 79
 *           300       61  0.00s               (gap is 73.67%)
 *           284      121  0.00s               (gap is 72.18%)
 *           281      181  0.00s               (gap is 71.89%)
 *           274      241  0.00s               (gap is 71.17%)
 *           267      301  0.00s               (gap 

In [47]:
sol = csp_transform_solution(msol, E_i, instance )
sol.cmax()

In [48]:
print(sol)

Objective : 234
Jobs sequence : 3	4	7	0	9	6	8	1	2	5
Machine_ID | Job_schedule (job_id , start_time , completion_time) | Completion_time
(3, 0, 14) : (4, 14, 46) : (7, 46, 56) : (0, 56, 87) : (9, 87, 109) : (6, 109, 128) : (8, 128, 140) : (1, 140, 173) : (2, 173, 184) : (5, 184, 202) | 202
(3, 14, 29) : (4, 46, 60) : (7, 60, 90) : (0, 90, 111) : (9, 111, 134) : (6, 134, 155) : (8, 155, 178) : (1, 178, 189) : (2, 189, 211) : (5, 211, 237) | 237
(3, 14, 45) : (4, 46, 79) : (7, 79, 98) : (0, 98, 123) : (9, 123, 145) : (6, 145, 171) : (8, 171, 185) : (1, 185, 206) : (2, 206, 226) : (5, 226, 234) | 234


In [56]:
machine = 2
list_events = sorted([(i, msol[M_k[machine][i]]) for i in E], key = lambda x: x[1][0])
print("\n".join(map(str,list_events)))

(3, IntervalVarValue(start=29, end=60, size=31))
(4, IntervalVarValue(start=60, end=93, size=33))
(7, IntervalVarValue(start=93, end=112, size=19))
(0, IntervalVarValue(start=112, end=137, size=25))
(9, IntervalVarValue(start=137, end=159, size=22))
(6, IntervalVarValue(start=159, end=185, size=26))
(8, IntervalVarValue(start=185, end=199, size=14))
(1, IntervalVarValue(start=199, end=220, size=21))
(2, IntervalVarValue(start=220, end=240, size=20))
(5, IntervalVarValue(start=240, end=248, size=8))


In [57]:
instance.to_txt("fs_instance.txt")

In [58]:
sol.to_txt("fs_solution.txt")